In [12]:
!pip install torchmetrics datasets tokenizers

In [13]:
from datasets import load_dataset
wmt14 = load_dataset('wmt14', 'de-en')

Found cached dataset wmt14 (/home/kydliceh/.cache/huggingface/datasets/wmt14/de-en/1.0.0/2de185b074515e97618524d69f5e27ee7545dcbed4aa9bc1a4235710ffca33f4)


  0%|          | 0/3 [00:00<?, ?it/s]

In [14]:
!git clone https://github.com/hynky1999/Statistical-learning-class
%cd /content/Statistical-learning-class/Assigments/Project

fatal: destination path 'Statistical-learning-class' already exists and is not an empty directory.
[Errno 2] No such file or directory: '/content/Statistical-learning-class/Assigments/Project'
/home/kydliceh/Projects/R/Assigments/Project


In [15]:
train_subset_length = 10
test_subset_length = len(wmt14['test'])
vocab_size=10

In [16]:
train_dataset = wmt14['train'].select(range(train_subset_length))
test_dataset = wmt14['test'].select(range(test_subset_length))

In [17]:
from tokenizers import ByteLevelBPETokenizer, Tokenizer



def create_tokenizer(iterable, add_special_tokens=False, train=True, vocab_size=1000):
    toknizer = ByteLevelBPETokenizer()
    if train:
        toknizer.train_from_iterator(iterable, vocab_size=vocab_size, min_frequency=2, special_tokens=["[PAD]"])

    if add_special_tokens:
        toknizer.add_special_tokens(["[START]", "[END]"])
    return toknizer


    

In [18]:

de_it = map(lambda x: x['de'] , train_dataset['translation'])
en_it = map(lambda x: x['en'] , train_dataset['translation'])

In [19]:
de_token = create_tokenizer(de_it, train=True, vocab_size=vocab_size, add_special_tokens=True)
en_token = create_tokenizer(en_it, train=True, vocab_size=vocab_size)


In [20]:
de_token.save("de_token.json")
en_token.save("en_token.json")

In [23]:

from tokenizers import ByteLevelBPETokenizer, Tokenizer

In [24]:
de_token = Tokenizer.from_file("de_token.json")
en_token = Tokenizer.from_file("en_token.json")

In [27]:
de_token.decode(de_token.encode("Hallo Welt").ids)

TypeError: 'str' object cannot be interpreted as an integer

In [ ]:
import torch
import numpy as np

In [ ]:
from torch.utils.data import DataLoader
def extract_embedding(embeds, lang):
    return {f"{lang}_ids": [e.ids for e in embeds], f"{lang}_att": [e.attention_mask for e in embeds]}

def add_lenghts(trans):
  translation = trans["translation"]
  de_sent = [t["de"] for t in translation]
  en_sent = [t["en"] for t in translation]
  en_len = [len(t["en"]) for t in translation]
  de_len = [len(t["de"]) for t in translation]
  return {"en_len": en_len, "de_len": de_len,"de_sent": de_sent, "en_sent": en_sent}



def tokenize(trans):
    translation = trans["translation"]
    de_sent = [t["de"] for t in translation]
    en_sent = [t["en"] for t in translation]
    en = en_token.encode_batch(en_sent)
    de = de_token.encode_batch(de_sent)
    dct = {**extract_embedding(en, "en"), **extract_embedding(de, "de")}
    return dct

def collate_fc(batch):
    en_ids_padded = torch.nn.utils.rnn.pad_sequence([b["en_ids"] for b in batch], batch_first=True, padding_value=en_token.token_to_id("[PAD]"))

    de_ids = [torch.cat([torch.tensor([de_token.token_to_id("[START]")]), b["de_ids"], torch.tensor([de_token.token_to_id("[END]")])]) for b in batch]

    de_ids_padded = torch.nn.utils.rnn.pad_sequence(de_ids, batch_first=True, padding_value=de_token.token_to_id("[PAD]"))

    en_sents = [b["en_sent"] for b in batch]
    de_sents = [b["de_sent"] for b in batch]

    return {"en_ids": en_ids_padded, "de_ids": de_ids_padded, "de_sent": de_sents, "en_sent": en_sents}

def create_dataloader(dataset, batch_size=32, shuffle=False):
    tokenized = dataset.map(add_lenghts, batch_size=batch_size, batched=True)
    # Sort by lengths to get smaller paddings
    tokenized = tokenized.sort("en_len")
    tokenized = tokenized.sort("de_len",kind="stable")
    tokenized = tokenized.map(tokenize, batch_size=batch_size, batched=True)
    tokenized = tokenized.remove_columns("translation")
    tokenized.set_format("torch")
    return DataLoader(tokenized, batch_size=batch_size, shuffle=True,collate_fn=collate_fc)



In [ ]:
next(iter(create_dataloader(train_dataset, batch_size=100)))["en_ids"].shape

NameError: name 'train_dataset' is not defined

In [ ]:
dataloader_train = create_dataloader(train_dataset, batch_size=2, shuffle=True)
dataloader_test = create_dataloader(test_dataset, batch_size=2, shuffle=True)

Loading cached processed dataset at /home/kydliceh/.cache/huggingface/datasets/wmt14/de-en/1.0.0/2de185b074515e97618524d69f5e27ee7545dcbed4aa9bc1a4235710ffca33f4/cache-822b218e8b90543c.arrow
Loading cached sorted indices for dataset at /home/kydliceh/.cache/huggingface/datasets/wmt14/de-en/1.0.0/2de185b074515e97618524d69f5e27ee7545dcbed4aa9bc1a4235710ffca33f4/cache-f6ac8a6d08304d29.arrow
Loading cached sorted indices for dataset at /home/kydliceh/.cache/huggingface/datasets/wmt14/de-en/1.0.0/2de185b074515e97618524d69f5e27ee7545dcbed4aa9bc1a4235710ffca33f4/cache-70c0be8a84b650d4.arrow
Loading cached processed dataset at /home/kydliceh/.cache/huggingface/datasets/wmt14/de-en/1.0.0/2de185b074515e97618524d69f5e27ee7545dcbed4aa9bc1a4235710ffca33f4/cache-cbed3d89d3c8f159.arrow


  0%|          | 0/1000 [00:00<?, ?ba/s]

  0%|          | 0/1000 [00:00<?, ?ba/s]

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from train_test import train, evaluate
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter

In [ ]:
from model import WMTModel
d_model=128
model = WMTModel(en_token.get_vocab_size(), de_token.get_vocab_size() , d_model)
model.to(device)
# Set to square root of model
# Then multiply by  min(step_num^−0.5 , step_num * warmup_steps^−1.5)
initial_lr = d_model ** -0.5
warmup_steps = 4000
multiplier_lambda = lambda step: min((step+1) ** -0.5, (step+1) * warmup_steps ** -1.5)
optimizer = torch.optim.Adam(model.parameters(), betas=(0.9, 0.98), eps=1e-9, lr=initial_lr)
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, multiplier_lambda)
criterion = nn.CrossEntropyLoss(ignore_index=0)
writer = SummaryWriter()
epochs=1000
save_every=2
for epoch in range(epochs):
    train(model, optimizer, scheduler ,criterion, dataloader_train, writer, epoch, minibatch=True, pad_token=en_token.token_to_id("[PAD]"))

    if epoch % save_every == 0:
        torch.save(model.state_dict(), f"model_{epoch}.pt")


Progress/train 0: 0/0 Loss: 5.643998622894287, Time: 0.39458537101745605
Progress/train 1: 0/0 Loss: 5.6983184814453125, Time: 0.4585902690887451
Progress/train 2: 0/0 Loss: 5.66721773147583, Time: 0.811173677444458
Progress/train 3: 0/0 Loss: 5.664276123046875, Time: 0.9107768535614014
Progress/train 4: 0/0 Loss: 5.620601654052734, Time: 0.39592552185058594


Exception ignored in: <function Dataset.__del__ at 0x7f76907724d0>
Traceback (most recent call last):
  File "/home/kydliceh/.conda/envs/NLP/lib/python3.10/site-packages/datasets/arrow_dataset.py", line 1248, in __del__
    def __del__(self):
  File "_pydevd_bundle/pydevd_cython.pyx", line 1443, in _pydevd_bundle.pydevd_cython.SafeCallWrapper.__call__
  File "_pydevd_bundle/pydevd_cython.pyx", line 1744, in _pydevd_bundle.pydevd_cython.ThreadTracer.__call__
  File "/home/kydliceh/.local/lib/python3.10/site-packages/debugpy/_vendored/pydevd/_pydev_bundle/pydev_is_thread_alive.py", line 9, in is_thread_alive
    def is_thread_alive(t):
KeyboardInterrupt: 


Progress/train 5: 0/0 Loss: 5.70603084564209, Time: 0.17795944213867188
Progress/train 6: 0/0 Loss: 5.681488513946533, Time: 0.757025957107544
Progress/train 7: 0/0 Loss: 5.744671821594238, Time: 0.19061541557312012
Progress/train 8: 0/0 Loss: 5.645264148712158, Time: 0.4288811683654785
Progress/train 9: 0/0 Loss: 5.626806259155273, Time: 0.6366860866546631
Progress/train 10: 0/0 Loss: 5.624164581298828, Time: 0.8401386737823486
Progress/train 11: 0/0 Loss: 5.6089019775390625, Time: 0.40249109268188477
Progress/train 12: 0/0 Loss: 5.661619663238525, Time: 0.20670270919799805
Progress/train 13: 0/0 Loss: 5.644193172454834, Time: 0.8352255821228027
Progress/train 14: 0/0 Loss: 5.590433597564697, Time: 0.6516189575195312
Progress/train 15: 0/0 Loss: 5.6176323890686035, Time: 0.6995832920074463
Progress/train 16: 0/0 Loss: 5.635136127471924, Time: 0.3603208065032959
Progress/train 17: 0/0 Loss: 5.705245494842529, Time: 0.3335585594177246
Progress/train 18: 0/0 Loss: 5.57806396484375, Time:

In [ ]:
evaluate(model, dataloader_test, writer, de_token)

KeyboardInterrupt: 